In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "운동복", "param": [ "50000818"]},
          {"name": "상의", "param": [ "50000803"]},
          {"name": "세트", "param": [ "50000816"]},
          #{"name": "수영복/비치웨어", "param": [ "50004870"]},
          {"name": "아우터", "param": [ "50000806"]},
          {"name": "언더웨어", "param": [ "50000168"]},
          {"name": "잠옷/홈웨어", "param": [ "50000826"]},
          {"name": "하의", "param": [ "50000808"]},
          {"name": "한복", "param": [ "50000819"]},
          {"name": "파티복", "param": [ "50000820"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "패션의류", "param": [ "50000000"]},
      {"name": "원피스", "param": [ "50000807"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "패션/의류")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "원피스")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["패션/의류"], axis=1, inplace=True)
df_all_a

,원피스,운동복,상의,세트,아우터,언더웨어,잠옷/홈웨어,하의,한복,파티복,유형,성별,연령
period,,,,,,,,,,,,,
2021-05-10,7.48706,1.03792,3.91988,0.18236,1.30093,3.82365,1.03295,1.47607,0.18031,0.25655,PC+MO,전체,전체
2021-05-11,7.47142,0.98962,4.00357,0.18822,1.24624,3.80137,1.05664,1.50960,0.17355,0.23965,PC+MO,전체,전체
2021-05-12,7.77586,1.02121,4.33135,0.19717,1.25771,3.86608,0.98787,1.65120,0.17830,0.22912,PC+MO,전체,전체
2021-05-13,8.26052,0.99859,4.56753,0.18664,1.17828,3.86226,1.04392,1.61937,0.16659,0.20737,PC+MO,전체,전체
2021-05-14,8.32612,0.90604,4.42689,0.17195,1.05615,3.90947,1.11255,1.56755,0.16052,0.20613,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,7.47958,0.51617,3.32641,0.09015,0.80862,3.35942,0.94094,1.35853,0.10286,0.15375,MO,전체,전체
2023-06-30,6.77746,0.46087,2.96931,0.08271,0.73089,3.02953,0.84589,1.23378,0.08815,0.12888,MO,전체,전체
2023-07-01,8.78491,0.61343,3.66284,0.10006,0.86368,3.73580,1.04481,1.53095,0.11476,0.15064,MO,전체,전체


In [4]:
## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "운동복", "param": [ "50000818"]},
          {"name": "상의", "param": [ "50000803"]},
          {"name": "세트", "param": [ "50000816"]},
          #{"name": "수영복/비치웨어", "param": [ "50004870"]},
          {"name": "아우터", "param": [ "50000806"]},
          {"name": "언더웨어", "param": [ "50000168"]},
          {"name": "잠옷/홈웨어", "param": [ "50000826"]},
          {"name": "하의", "param": [ "50000808"]},
          {"name": "한복", "param": [ "50000819"]},
          {"name": "파티복", "param": [ "50000820"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "패션의류", "param": [ "50000000"]},
      {"name": "원피스", "param": [ "50000807"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "패션/의류")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "원피스")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["패션/의류"], axis=1, inplace=True)
df_all_b

,원피스,운동복,상의,세트,아우터,언더웨어,잠옷/홈웨어,하의,한복,파티복,유형,성별,연령
period,,,,,,,,,,,,,
2021-05-10,9.46650,1.26823,4.45950,0.23140,1.67314,4.70721,1.25919,1.88994,0.21303,0.31610,PC+MO,여성,전체
2021-05-11,9.41897,1.20958,4.60237,0.22954,1.60495,4.68557,1.29381,1.92951,0.20651,0.29830,PC+MO,여성,전체
2021-05-12,9.90263,1.25859,5.03690,0.24158,1.61470,4.74998,1.19130,2.10937,0.21197,0.28870,PC+MO,여성,전체
2021-05-13,10.56736,1.22781,5.30422,0.22866,1.51926,4.77043,1.27381,2.06762,0.19687,0.25833,PC+MO,여성,전체
2021-05-14,10.62680,1.11147,5.13726,0.21078,1.35658,4.80569,1.35995,2.00015,0.18535,0.25371,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,10.32706,0.67646,4.48340,0.12321,1.12520,4.50998,1.26075,1.89652,0.12664,0.20322,MO,여성,전체
2023-06-30,9.36085,0.60101,3.97946,0.11355,1.02137,4.05407,1.12963,1.72715,0.10973,0.16664,MO,여성,전체
2023-07-01,12.13682,0.80123,4.96249,0.13872,1.19879,5.04304,1.41064,2.15026,0.14441,0.19811,MO,여성,전체


In [5]:
## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "운동복", "param": [ "50000818"]},
          {"name": "상의", "param": [ "50000803"]},
          {"name": "세트", "param": [ "50000816"]},
          #{"name": "수영복/비치웨어", "param": [ "50004870"]},
          {"name": "아우터", "param": [ "50000806"]},
          {"name": "언더웨어", "param": [ "50000168"]},
          {"name": "잠옷/홈웨어", "param": [ "50000826"]},
          {"name": "하의", "param": [ "50000808"]},
          {"name": "한복", "param": [ "50000819"]},
          {"name": "파티복", "param": [ "50000820"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "패션의류", "param": [ "50000000"]},
      {"name": "원피스", "param": [ "50000807"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "패션/의류")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "원피스")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["패션/의류"], axis=1, inplace=True)
df_all_c

,원피스,운동복,상의,세트,아우터,언더웨어,잠옷/홈웨어,하의,한복,파티복,유형,성별,연령
period,,,,,,,,,,,,,
2021-05-10,1.26525,0.39559,1.04222,0.02881,0.23763,1.18237,0.36069,0.23109,0.05938,0.07011,PC+MO,남성,전체
2021-05-11,1.24428,0.38008,1.03365,0.03345,0.21775,1.19035,0.35068,0.22226,0.05140,0.06520,PC+MO,남성,전체
2021-05-12,1.21732,0.37499,1.10621,0.03584,0.21252,1.22367,0.34081,0.25445,0.05388,0.05758,PC+MO,남성,전체
2021-05-13,1.23414,0.35965,1.16429,0.02673,0.18307,1.19107,0.34820,0.23236,0.05623,0.05555,PC+MO,남성,전체
2021-05-14,1.29857,0.31393,1.09101,0.02570,0.17333,1.16330,0.33918,0.24065,0.05605,0.05208,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,1.17683,0.19034,0.87744,0.01641,0.12828,1.00958,0.29236,0.22970,0.03891,0.04777,MO,남성,전체
2023-06-30,1.03619,0.16408,0.77923,0.01355,0.11468,0.91740,0.25814,0.19588,0.02888,0.04702,MO,남성,전체
2023-07-01,1.38384,0.23782,0.99445,0.01725,0.15384,1.04984,0.29182,0.22066,0.04272,0.05133,MO,남성,전체


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)

## 엑셀에 저장
df_all.to_excel("MCLS_09_all_fm_all.xlsx", sheet_name="click ratio")
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   원피스     7065 non-null   float64
 1   운동복     7065 non-null   float64
 2   상의      7065 non-null   float64
 3   세트      7065 non-null   float64
 4   아우터     7065 non-null   float64
 5   언더웨어    7065 non-null   float64
 6   잠옷/홈웨어  7065 non-null   float64
 7   하의      7065 non-null   float64
 8   한복      7065 non-null   float64
 9   파티복     7065 non-null   float64
 10  유형      7065 non-null   object 
 11  성별      7065 non-null   object 
 12  연령      7065 non-null   object 
dtypes: float64(10), object(3)
memory usage: 772.7+ KB
